In [0]:
import datetime
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType,StructField,DateType,TimestampType,StringType,IntegerType,DecimalType
from pyspark.sql.functions import year, month, dayofmonth,to_timestamp,to_date,split,substring,col,when,lit
from pyspark.sql.functions import udf

In [0]:
def replaceNullCustomerID(df):
  df = df.na.fill('Guest',subset = ["Customer ID"])
  return df

def replaceNullDescription(df):
  df = df.na.fill('Unlisted',subset = ["Description"])
  return df

def addcolumnQuarter(df):
  split_col = split(df['InvoiceDate'], '/')
  df = df.withColumn("Year",substring(split_col.getItem(2),1,4))
  df = df.withColumn('month',split_col.getItem(0).cast("int"))
  df = df.withColumn('Qtr',(when( (col("month") == 1) | (col("month") == 2) | (col("month") == 3),"Qtr1")
                           .when( (col("month") == 4) | (col("month") == 5) | (col("month") == 6),"Qtr2")
                           .when( (col("month") == 7) | (col("month") == 8) | (col("month") == 9) ,"Qtr3")
                           .otherwise("Qtr4")))
  df = df.drop(col("Year"))
  df = df.drop(col("month"))
  
  return df
  
def addcolumnInvoiceType(df):
  df = df.withColumn('InvoiceType',(when( (col("Quantity") <= 0),"Return")
                         .otherwise("Purchase")))
  return df

def filterDf(df):
  df_uk = df.filter(df.Country == 'United Kingdom')
  df_others = df.filter(df.Country != 'United Kingdom')
  return df_uk,df_others